# FGSM Adversarial Attack - Interactive Lab

This notebook demonstrates FGSM attacks interactively.

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision.utils import save_image
import imageio
import os
from ipywidgets import interact, FloatSlider
import numpy as np
from PIL import Image

In [ ]:
# Load MNIST Dataset
transform = transforms.Compose([transforms.ToTensor()])
testset = torchvision.datasets.MNIST(root='../datasets', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Simple CNN Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        return self.fc(x)

model = Net().to(device)
model_path = "../models/mnist_model.pth"
os.makedirs("../models", exist_ok=True)

if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
else:
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    trainset = torchvision.datasets.MNIST(root='../datasets', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
    model.train()
    for epoch in range(1):
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    torch.save(model.state_dict(), model_path)

model.eval()

In [ ]:
# FGSM Attack
def fgsm_attack(image, epsilon, data_grad):
    perturbed_image = image + epsilon * data_grad.sign()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

In [ ]:
# Interactive Attack Visualization
def attack_visual(epsilon):
    dataiter = iter(testloader)
    image, label = next(dataiter)
    image, label = image.to(device), label.to(device)

    image.requires_grad = True
    output = model(image)
    init_pred = output.max(1, keepdim=True)[1]

    loss = nn.CrossEntropyLoss()(output, label)
    model.zero_grad()
    loss.backward()
    data_grad = image.grad.data

    perturbed_image = fgsm_attack(image, epsilon, data_grad)

    # Plotting original vs perturbed
    fig, axs = plt.subplots(1, 3, figsize=(12,4))
    axs[0].imshow(image.squeeze().cpu().detach().numpy(), cmap="gray")
    axs[0].set_title("Original")
    axs[0].axis('off')

    axs[1].imshow((perturbed_image.squeeze().cpu().detach().numpy()), cmap="gray")
    axs[1].set_title("Adversarial")
    axs[1].axis('off')

    axs[2].imshow((perturbed_image - image).squeeze().cpu().detach().numpy(), cmap='hot')
    axs[2].set_title("Perturbation Heatmap")
    axs[2].axis('off')
    
    plt.tight_layout()
    plt.show()

interact(attack_visual, epsilon=FloatSlider(min=0, max=0.5, step=0.01, value=0.1));